<a href="https://colab.research.google.com/github/dasmiq/cs6200-retrieval-models/blob/main/retrieval-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Models and Evaluation



In this notebook, you will evaluate results ranking on a test collection. First, you'll compute the mean average precision of a baseline BM25 model. Then you'll implement a query-likelihood retrieval model based on the same inverted index.

This notebook uses the [Pyserini](http://pyserini.io/) library, a Python interface to [Anserini](http://anserini.io) and thus to [Lucene](https://lucene.apache.org/), a widely-used open-source search engine. This library is written and maintained by Jimmy Lin and his colleagues at the University of Waterloo.


We start by installing the python interface. Since it calls the underlying Lucene search engine, which is written in Java, we make sure we point to an appropriate Java installation. If like Colab you don't have Java 21, uncomment the following code and run it, or whatever makes sense for your platform.

In [1]:
## Uncomment the following code to install Java 21 on Colab
#apt-get install openjdk-21-jre-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk"
!java -version

openjdk version "22.0.1-internal" 2024-04-16
OpenJDK Runtime Environment (build 22.0.1-internal-adhoc.conda.src)
OpenJDK 64-Bit Server VM (build 22.0.1-internal-adhoc.conda.src, mixed mode, sharing)


In [2]:
#!pip install pyserini
# You can change this to gpu if you have one.
# It's a pyserini dependency, but we won't need it until the next assignment.
#!pip install faiss-cpu

You can use the `LuceneSearcher` to search over an index. We can initialize the searcher with a pre-built index, which Pyserini will automatically download if it hasn't already. Note that the index takes up 1.6GB of disk.

In [3]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('robust04')

/home/redbear/miniforge3/envs/ds6220/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Mar 03, 2025 9:36:57 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


Now we can search for a query and inspect the results:

In [4]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [5]:
from pyserini.index import LuceneIndexReader
from IPython.core.display import display, HTML

reader = LuceneIndexReader.from_prebuilt_index('robust04')

doc = reader.doc('LA092790-0015').raw()
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

/tmp/ipykernel_30523/1735331188.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [6]:
analyzed = reader.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

The index also stores the raw document vector, which we can obtain as a Python dictionary of analyzed terms to counts (i.e., term frequency).
For brevity, we only look at terms that appear more than once:

In [7]:
doc_vector = reader.get_document_vector('LA092790-0015')
{ k: v for (k, v) in doc_vector.items() if v >1 }

{'been': 11,
 'habitat': 2,
 'year': 3,
 'jenk': 4,
 'mountain': 3,
 'rees': 3,
 'would': 2,
 'debusscher': 3,
 'near': 3,
 'hungri': 3,
 'past': 2,
 'especi': 2,
 'wildlif': 2,
 'lion': 3,
 'ventura': 4,
 'anim': 9,
 'forest': 2,
 'two': 3,
 'seen': 3,
 'attack': 3,
 'black': 2,
 'i': 2,
 'food': 3,
 'counti': 4,
 'rural': 3,
 "they'r": 5,
 'sever': 3,
 'california': 2,
 'advis': 2,
 'parch': 2,
 'area': 4,
 'she': 3,
 'month': 4,
 'deer': 3,
 'famili': 2,
 'we': 2,
 'peopl': 2,
 'just': 3,
 'live': 2,
 'drought': 4,
 'ha': 3,
 'he': 2,
 'hi': 2,
 'septemb': 2,
 'yard': 2,
 'elsewher': 2,
 'natur': 2,
 'out': 4,
 'lot': 2,
 'have': 16,
 'leav': 2,
 'more': 3,
 'off': 2,
 'ojai': 3,
 'report': 4,
 'bobcat': 4,
 'coyot': 10,
 'few': 2,
 'bear': 4,
 'depart': 2,
 'author': 3,
 'dry': 2,
 'vallei': 4,
 'who': 2,
 'game': 2,
 'resid': 2,
 'hous': 3,
 'cat': 3,
 'sai': 2,
 'said': 19,
 'park': 2,
 'on': 2,
 'offici': 5,
 'wild': 3}

## Evaluating Ranked Results

We can load some standard evaluation sets such as Robust04, which contains 250 queries, or "topics" as the TREC conferences call them.

In [8]:
from pyserini.search import get_topics
topics = get_topics('robust04')
print(f'{len(topics)} queries total')

250 queries total


The topics are in a dictionary, whose keys are integers uniquely identifying each query. Each topic contains the following fields:

* `title`: TREC's term for the brief query a user might actually type;
* `description`: a longer form of the query in the form of a complete sentence; and
* `narrative`: a description of what the user is looking for and what kinds of results would be relevant or non-relevant.

In [9]:
topics[301]

{'narrative': 'A relevant document must as a minimum identify the organization and the type of illegal activity (e.g., Columbian cartel exporting cocaine). Vague references to international drug trade without identification of the organization(s) involved would not be relevant.',
 'description': 'Identify organizations that participate in international criminal activity, the activity, and, if possible, collaborating organizations and the countries involved.',
 'title': 'International Organized Crime'}

For the purpose of your experiments, we'll divide them into a development and test set.

In [10]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

Now, we'll fetch the relevance judgments for the Robust04 queries, which TREC calls "qrels".

In [11]:
from urllib.request import urlopen

qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
qrels = []
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  qrels.append([int(qid), 0, docid.decode('UTF-8'), int(score)])
#qrels = [line.strip().split() for line in urlopen(qfile)]

Each record in the qrel contains four fields:

1. the numeric identifier of the query;
2. the round of relevance feedback, which is here always 0;
3. the identifier of a documennt that has been judged; and
4. the relevance score of that document.

In Robust04, all relevance judgments are binary, i.e., 1 or 0. Note that not all non-relevant documents are recorded. The qrel file only contains those documents the annotators actually looked at; the vast majority of documents in the collection have not been judged. In IR evaluation, we assume that unannotated documents are non-relevant.

In [12]:
qrels[:10]

[[301, 0, 'FBIS3-10082', 1],
 [301, 0, 'FBIS3-10169', 0],
 [301, 0, 'FBIS3-10243', 1],
 [301, 0, 'FBIS3-10319', 0],
 [301, 0, 'FBIS3-10397', 1],
 [301, 0, 'FBIS3-10491', 1],
 [301, 0, 'FBIS3-10555', 0],
 [301, 0, 'FBIS3-10622', 1],
 [301, 0, 'FBIS3-10634', 0],
 [301, 0, 'FBIS3-10635', 0]]

## Computing Mean Average Precision

The Robust04 collection uses binary relevance judgments and usually has multiple relevant results for each query. It is thus common to use **mean average precision** (MAP) to evaluate retrieval performance on it. Remember from class that MAP adds the precision at the position of each _relevant_ document in a ranked list and then divides by the total number of relevant documents. So that we don't have to scan through the entire collection, we usually evaluate MAP at some maximum rank value, such as 100 or 1000. We simply stop scanning at that maximum rank.

As we saw above, you should pass a query string (the `title` of a topic) and the desired number of results to the `search` method of the `searcher` object.

For this assignment, evaluate MAP@1000 for the list of `test_topics` we created above. You should process the `qrels` data to find the relevant results for each query.

In [13]:
## TODO [35 points]: Compute MAP@1000 for test_topics
# first process the qrels into a dictionary, where key is the query id and value is a list of dictionaries

from tqdm import tqdm
import numpy as np
qrels_dict = {}
for qid, round, docid, score in qrels:
    if qid not in qrels_dict:
        qrels_dict[qid] = []
    qrels_dict[qid].append({'docid': docid, "score": score})

def search_topic(topic_text, searcher, k=1000):
    hits = searcher.search(topic_text, k)
    docids = [hit.docid for hit in hits]
    scores = [hit.score for hit in hits]
    return docids, scores

def compute_ap(qrels_list, docids):
    """Computes the average precision for a single query given the qrels and the retrieved documents.

    Args:
        qrels (list): A list of tuples in the format (topic_id, 0, doc_id, relevance)
        docids (list): A list of retrieved documents based on the query
    """
    
    if docids is None or len(docids) == 0 or qrels_list is None or len(qrels_list) == 0:
        return .0
    
    relevant_docs = {qrel['docid'] for qrel in qrels_list if qrel['score'] >= 1} 
    # all of the docs that are not in the relevant docs are irrelevant
    
    docids = np.array(docids)
    
    relevance_mask = np.isin(docids, list(relevant_docs))
    
    # compute the number of relevant docs at each rank
    num_relevant_docs = np.cumsum(relevance_mask)
    
    # compute the precision at each rank
    precision = num_relevant_docs / np.arange(1, len(docids) + 1)
    
    # Sum up only the precision scores where a relevant document was found
    ap = np.sum(precision[relevance_mask]) / len(relevant_docs)
    
    return ap


def compute_map(topics, searcher, qrels_dict, k=1000):
    ap_scores = []
    for qid, query in tqdm(topics.items()):
        retrieved_docs, _ = search_topic(query['title'], searcher)
        ap = compute_ap(qrels_dict.get(qid, []), retrieved_docs)
        ap_scores.append(ap)
    return sum(ap_scores) / len(ap_scores)

map_1000_bm25 = compute_map(test_topics, searcher, qrels_dict)
print(f'MAP@1000: {map_1000_bm25:.4f}')

100%|██████████| 125/125 [00:05<00:00, 23.20it/s]

MAP@1000: 0.2616


## Implementing a Query Likelihood Model

The default `LuceneSearcher` in pyserini uses a BM25 model. In this part of the assignment, you'll interact directly with the index to implement a query likelihood retrieval model.  For more details on pyserini's index API, [see the documentation](https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md).

We've already created an `IndexReader` for the Robust04 collection and assigned it to `reader`. Remember that we can pass documents or queries through the index's predefined analysis pipeline, a series of operations like stemming, stopword removal, etc.

In [14]:
reader.analyze('Are there black bear attacks there?')

['black', 'bear', 'attack']

For most retrieval models, including BM25 and query likelihood, we'll want to know the overall statistics for the terms we retrieve from the index. By default, pyserini will do stemming and other "analysis" steps on the terms you give it. But since we're already working with stemmed queries, we'll tell it not to perform this analysis now.

In [15]:
# Just checking that our term is already in its index form.
print(reader.analyze('bear'))

df, cf = reader.get_term_counts('bear', analyzer=None)
print(f'The document frequency is {df} and the collection frequency is {cf}.')

['bear']
The document frequency is 16429 and the collection frequency is 23545.


You might also want to know summary statistics about the collection.

In [16]:
reader.stats()

{'total_terms': 174540872,
 'documents': 528030,
 'non_empty_documents': 528030,
 'unique_terms': 923436}

Now let's traverse the postings for a term.

In [17]:
posting_list = reader.get_postings_list('bear', analyzer=None)
print(f'There are {len(posting_list)} postings.')

posting = posting_list[100]
print(f'Each posting contains a document ID ({posting.docid}), term frequency ({posting.tf}), and position list ({posting.positions}).')

print(f'searcher has a method for turning internal integer IDs like {posting.docid} into external IDs like {searcher.doc(posting.docid).docid()}.')

There are 16429 postings.
Each posting contains a document ID (3405), term frequency (1), and position list ([461]).
searcher has a method for turning internal integer IDs like 3405 into external IDs like LA031889-0009.


If, instead of mapping terms to documents, you want to use a document name to get term information, see above for the `get_document_vector` method.

Finally, it will be useful to have some information about each document in the collection, which is saved in a file in the GitHub repository.

In [18]:
import json
docfile = urlopen('https://github.com/dasmiq/cs6200-retrieval-models/raw/refs/heads/main/doclen.jsonl')
docinfo = [json.loads(line) for line in docfile]

Each record in that file contains the document `id`, its interned numeric `iid`, and the number of tokens `n`.

In [19]:
len(docinfo)

528030

Use Dirichlet smoothing with $\mu$ equal to the mean document length in the collection.

In [20]:
# TODO [2 points]: Compute the mean document length from collection statistics.
mu = np.sum([doc['n'] for doc in docinfo]) / len(docinfo)
print(f'The mean document length is {mu:.2f}.')

The mean document length is 330.55.


Now you should have all the information you need to implement a query likelihood retrieval model.

In [21]:
## TODO [50 points]: Implement a query likelihood model with Dirichlet smoothing.
## For a given query, index reader, and constant k, return the k documents with the highest query likelihoods.
## Think about what data structure would let you keep the top k.

# in order to maintain top k values I am going to use a sorted list
# at each time I will insert a new value to keep the list sorted, I will check if the length of the list is greater than k
# if it is I will remove the first element 

# I am going to use the library bisect to insert the new value in the correct position

import bisect # https://docs.python.org/3/library/bisect.html


def dirchlet_search(query, reader, docinfo = docinfo, k=1000):
    query_terms = reader.analyze(query)
    top_k = []
    mu = np.sum([doc['n'] for doc in docinfo]) / len(docinfo)
    
    # define a list of docs that we want to go over, it should contain at least a single value in the query
    docs = set()
    term_doc_frequencies = {}
    term_c_frequencies = {}
    #print("Retrieving the term frequencies")
    for term in query_terms:
        posting_list = reader.get_postings_list(term, analyzer=None)
        term_c_frequencies[term] = reader.get_term_counts(term, analyzer=None)[1]
        term_doc = {}
        for posting in posting_list:
            docs.add(posting.docid)
            term_doc[posting.docid] = posting.tf
        term_doc_frequencies[term] = term_doc
            
    
    #print("Computing likelihoods")
    top_k = []
    C = reader.stats()['total_terms'] # total number of terms in collection
    for docid in docs:
        single_docinfo = docinfo[docid]
        D = single_docinfo['n'] # number of terms in the document
        score = 0
        for term in query_terms:
            cf = term_c_frequencies[term] # get the document and collection frequency
            # get the term frequency for the term in the document
            tf = term_doc_frequencies[term].get(docid, 0)
            # compute the log likelihood
            score += np.log((tf + mu * cf / C)/ (D + mu))
            
        # get the document external id because this is how the searcher is returning it
        docid_external = searcher.doc(docid).docid()
        # insert the score in the correct position
        bisect.insort(top_k, (docid_external, score), key=lambda x: x[1])
        # remove the first element if the length is greater than k
        if len(top_k) > k:
            top_k.pop(0)
    top_k.reverse()
    # return in the same format as reader.search function
    
    return top_k

            
# testing the function
top_k = dirchlet_search('black bear attacks', reader, docinfo, 1000)
print(len(top_k))
for docid, score in top_k[:10]:
    print(f'{docid} {score:.4f}')


1000
LA081689-0039 -16.3315
LA092790-0015 -16.5745
FBIS4-16530 -16.6780
LA102589-0076 -17.2878
FT932-15491 -17.4790
FBIS3-12276 -17.5706
LA091090-0085 -17.6674
FT931-736 -17.7476
FT922-13519 -17.8078
LA122889-0122 -17.8343


Now you can reuse some of your evaluation code for mean average precision from above.

In [22]:
## TODO [13 points]: Compute the MAP@1000 of your query likelihood model on the test_topics set.
from dataclasses import dataclass

# create a dummy searcher class that has only search method so I can reuse the compute_map function
@dataclass
class DocScore:
    docid: str
    score: float

class DummySearcher:
    def __init__(self, reader, docinfo):
        self.reader = reader
        self.docinfo = docinfo
    
    def search(self, query, k):
        temp_res = dirchlet_search(query, self.reader, self.docinfo, k)
        # convert the results to the same format as the searcher
        return [DocScore(docid, score) for docid, score in temp_res]

dummy_searcher = DummySearcher(reader, docinfo)

map_1000 = compute_map(test_topics, dummy_searcher, qrels_dict)
print(f'MAP@1000: {map_1000:.4f}')

100%|██████████| 125/125 [05:30<00:00,  2.64s/it]

MAP@1000: 0.2579


In [23]:
print(map_1000_bm25)

0.26161264686447017
